## Import Library and dataset

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from tensorflow.keras.regularizers import l2
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
df = pd.read_csv("../data/interests_without_onehot.csv")

features = df.iloc[:, :-1].values
labels = df["y"].values

dataset = tf.data.Dataset.from_tensor_slices((features, labels))

In [3]:
for features, label in dataset.take(5):
    print("features :", features)
    print("Label:", label)
    print("Label dtype:", label.dtype)

features : tf.Tensor([0 1 0 2 1 0 0 2 0 1 0 2 0 0 1 1 0 0 2 1 0 1 2 1 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 2 2 1 1 1 0 2 0 1 2 1 0], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([2 1 2 0 0 1 2 2 0 1 2 0 0 1 2 1 0 0 2 1 0 2 0 1 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([0 2 0 0 0 2 2 0 0 0 1 1 0 0 1 1 0 0 2 1 0 2 2 0 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>
features : tf.Tensor([1 1 1 1 1 1 1 1 0 1 2 1 0 1 1 1 1 1 0 1 2 1 1 1 1], shape=(25,), dtype=int64)
Label: tf.Tensor(b'IT sector', shape=(), dtype=string)
Label dtype: <dtype: 'string'>


## Preprocessing Dataset

In [4]:
keys = tf.constant(["IT sector", "Goverment sector", "Health sector", "Education sector", "Sports sector", "Finance sector", "Entertainment sector"])
values = tf.constant([0, 1, 2, 3, 4, 5, 6])

table = tf.lookup.StaticHashTable(
    initializer=tf.lookup.KeyValueTensorInitializer(keys, values),
    default_value=-1
)

def preprocess(features, label):
    label_int = table.lookup(label)
    label_encoded = tf.one_hot(label_int, depth=7)
    
    return features, label_encoded

dataset = dataset.map(preprocess)

In [5]:
for features, labels in dataset.take(5):
    print("Original Features:", features)
    print("One-Hot Encoded Labels:", labels.numpy())

Original Features: tf.Tensor([0 1 0 2 1 0 0 2 0 1 0 2 0 0 1 1 0 0 2 1 0 1 2 1 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 0 1 0 0 0 0 0 0 0 0 0 0 2 2 1 1 1 0 2 0 1 2 1 0], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([2 1 2 0 0 1 2 2 0 1 2 0 0 1 2 1 0 0 2 1 0 2 0 1 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([0 2 0 0 0 2 2 0 0 0 1 1 0 0 1 1 0 0 2 1 0 2 2 0 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]
Original Features: tf.Tensor([1 1 1 1 1 1 1 1 0 1 2 1 0 1 1 1 1 1 0 1 2 1 1 1 1], shape=(25,), dtype=int64)
One-Hot Encoded Labels: [1. 0. 0. 0. 0. 0. 0.]


## Splitting Dataset

In [6]:
features = []
labels = []

for feature, label in dataset:
    features.append(feature.numpy())
    labels.append(label.numpy())

features = np.array(features)
labels = np.array(labels)

X_train, X_test, y_train, y_test = train_test_split(
    features, labels, test_size=0.25, random_state=42, stratify=labels
)


train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train)).batch(32).prefetch(tf.data.AUTOTUNE)
test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(32).prefetch(tf.data.AUTOTUNE)

In [7]:
for features, labels in train_dataset.take(1):
    print("Features shape:", features.shape)
    print("Labels shape:", labels.shape)

Features shape: (32, 25)
Labels shape: (32, 7)


## Creating Model

In [73]:
model = Sequential([
    Dense(128, activation='relu', input_shape=(25,), kernel_regularizer=l2(0.01)),
    Dropout(0.4),
    Dense(64, activation='relu'),
    Dropout(0.3),
    Dense(32, activation='relu'),
    Dense(7, activation='softmax')
])

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

## Training Model

In [74]:
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True
)

history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=300,
    callbacks=[early_stopping]
)

Epoch 1/300
5/5 [==============================] - 2s 123ms/step - loss: 2.3746 - accuracy: 0.1678 - val_loss: 2.2888 - val_accuracy: 0.2200
Epoch 2/300
5/5 [==============================] - 0s 9ms/step - loss: 2.2676 - accuracy: 0.2349 - val_loss: 2.2062 - val_accuracy: 0.4400
Epoch 3/300
5/5 [==============================] - 0s 8ms/step - loss: 2.2181 - accuracy: 0.2953 - val_loss: 2.1253 - val_accuracy: 0.5800
Epoch 4/300
5/5 [==============================] - 0s 10ms/step - loss: 2.0939 - accuracy: 0.4027 - val_loss: 2.0431 - val_accuracy: 0.6200
Epoch 5/300
5/5 [==============================] - 0s 11ms/step - loss: 2.0558 - accuracy: 0.3960 - val_loss: 1.9573 - val_accuracy: 0.6200
Epoch 6/300
5/5 [==============================] - 0s 12ms/step - loss: 1.9776 - accuracy: 0.4430 - val_loss: 1.8632 - val_accuracy: 0.6400
Epoch 7/300
5/5 [==============================] - 0s 9ms/step - loss: 1.9038 - accuracy: 0.4698 - val_loss: 1.7596 - val_accuracy: 0.6800
Epoch 8/300
5/5 [=====

In [75]:
hasil = model.predict([[2,0,1,1,1,2,1,0,1,1,2,0,1,2,1,0,0,1,1,2,1,1,0,1,0]])
print(f"HASIL : {hasil}")

1/1 [==============================] - 0s 265ms/step
HASIL : [[5.5836010e-01 3.0170161e-01 6.6796239e-03 1.1631828e-02 4.1995905e-04
  1.7095628e-04 1.2103600e-01]]


## Save Model

In [76]:
model.save(filepath="../src/model/sector_classification_model.h5")